In [3]:
import pandas as pd
from PublicDataReader import Fred
import re
import time
import us

# FRED API 
api_key = "635a22d620ae98ebdba64c9a957ade95"

api = Fred(api_key)

## Prepairing

In [4]:
#pip install us

# ANSI to USPS with DC fix
def ansi_to_usps(ansi_code):
    if ansi_code == "11":  # Special case for DC
        return us.states.DC.abbr
    state = us.states.lookup(ansi_code)
    return state.abbr if state else None

def usps_to_names(usps_code):
    if usps_code == "DC":
        return us.states.DC.name
    state = us.states.lookup(usps_code)
    return state.name if state else None

In [5]:
CFS_counties = pd.read_excel('list2017.xlsx', 'CFSAreas', converters={'ST': str})

In [6]:
CFS_counties['State_USPS'] = CFS_counties['ST'].astype(str).str.zfill(2).apply(ansi_to_usps)
CFS_counties['State_Name'] = CFS_counties['State_USPS'].apply(usps_to_names)

In [7]:
CFS_counties.columns

Index(['ST', 'CNTY', 'CNTY_NAME', 'CFS07_AREA', 'CFS12_AREA', 'CFS17_AREA',
       'CFS07_TYPE', 'CFS12_TYPE', 'CFS17_TYPE', 'CFS07_DDESTGEO',
       'CFS07_GEOID', 'CFS12_GEOID', 'CFS17_GEOID', 'CFS07_NAME', 'CFS12_NAME',
       'CFS17_NAME', 'State_USPS', 'State_Name'],
      dtype='object')

In [8]:
def get_series_id(api, search_text, title_pattern, frequency, units):
    """Get series_id based on search criteria."""
    try:
        time.sleep(0.5)
        result = api.get_data(api_name="series_search", search_text=search_text)
        if not isinstance(result, pd.DataFrame) or result.empty:
            #print(f"Data not found for {search_text}")
            return None
        
        filtered_result = result[
            result['title'].str.contains(title_pattern, na=False, case=False) &
            (result['frequency'] == frequency) &
            (result['units'] == units)
        ]
        
        if not filtered_result.empty:
            return filtered_result.iloc[0]['id']
        else:
            #print(f"Data not found for {county_name}, {state_code}")
            return None

    except Exception as e:
        #print(f"Error processing {county_name}, {state_code}: {e}")
        return None

def get_data(api, series_id):
    """Get data for a given series_id."""
    if pd.isna(series_id) or series_id == "None":
        return None
    time.sleep(0.5)
    df = api.get_data(api_name="series_observations", series_id=series_id)
    if not isinstance(df, pd.DataFrame) or df.empty:
        return None
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df = df[(df['date'].dt.year >= 2018) & (df['date'].dt.year <= 2023)]
    df.set_index('date', inplace=True)
    return df if not df.empty else None

## Population

In [9]:
def get_series_id_pop(api, county_name, state_code):
    """Get series_id for county-level population data."""
    search_text = f"{county_name}, {state_code} Resident Population"
    title_pattern = f"^Resident Population in (?:the )?{county_name}, {state_code}(?:\\s*\\(.*\\))?$"
    return get_series_id(api, search_text, title_pattern, 'Annual', 'Thousands of Persons')

def get_series_id_state_pop(api, state_name):
    """Get series_id for state-level population data."""
    search_text = f"{state_name} Resident Population"
    title_pattern = f"^Resident Population in (?:the )?{state_name}$"
    return get_series_id(api, search_text, title_pattern, 'Annual', 'Thousands of Persons')

def add_series_ids_pop(api, CFS_counties, max_retries=5):
    """Assign population series_id with manual overrides for specific counties."""
    
    # Manual series for "search-problematic" counties
    MANUAL_SERIES = {
        ('LaSalle County', 'IL'): 'ILLASA0POP',
        ('Broomfield County', 'CO'): 'COBROO4POP',
        ('Denver County', 'CO'): 'CODENV5POP',
        ('Philadelphia County', 'PA'): 'PAPHIL5POP',
        ('San Francisco County', 'CA'): 'CASANF0POP',
        ('Honolulu County', 'HI'): 'HIHONO7POP',
        ('District of Columbia', 'DC'): 'DCPOP'
    }

    CFS_counties = CFS_counties.copy()
    CFS_counties['series_id_pop'] = None

    for idx, row in CFS_counties.iterrows():
        if "Remainder of" in row['CFS17_NAME']:
            continue  # Skip
        
        key = (row['CNTY_NAME'], row['State_USPS'])
        
        if key in MANUAL_SERIES:
            CFS_counties.at[idx, 'series_id_pop'] = MANUAL_SERIES[key]
        else:
            series_id = None
            for _ in range(max_retries):
                series_id = get_series_id_pop(api, row['CNTY_NAME'], row['State_USPS'])
                if series_id:
                    break
                time.sleep(0.5)  # Wait before retry
            
            CFS_counties.at[idx, 'series_id_pop'] = series_id

    return CFS_counties

In [10]:
def get_state_pop(api, CFS_counties, max_retries=3):
    """Get population data for each state for each year and store it in a separate DataFrame."""
    state_pop_data = []

    for state_name in CFS_counties['State_Name'].unique():
        series_id = get_series_id_state_pop(api, state_name)
        if not series_id:
            continue  # Skip, if there is no series_id
        
        pop_data = None
        for _ in range(max_retries):
            pop_data = get_data(api, series_id)
            if pop_data is not None:
                break
            time.sleep(0.5)

        if pop_data is not None:
            state_pop_data_row = {'State_Name': state_name}
            for year in range(2018, 2024):
                value = pop_data.loc[pop_data.index.year == year, 'value']
                state_pop_data_row[f'POP_{year}'] = value.iloc[0]/1000 if not value.empty else None
            state_pop_data.append(state_pop_data_row)

    return pd.DataFrame(state_pop_data)


def add_pop_to_counties(api, CFS_counties, max_retries=5):
    """Get population data with forward fill for missing years in Connecticut counties."""
    CFS_counties = CFS_counties.copy()
    years = list(range(2018, 2024))

    for year in years:
        CFS_counties[f'POP_{year}'] = None

    CONNECTICUT_COUNTIES = {
        'Fairfield County', 'Litchfield County', 'New Haven County',
        'Hartford County', 'Middlesex County', 'Tolland County'
    }

    for idx, row in CFS_counties.iterrows():
        if pd.notna(row['series_id_pop']):
            pop_data = None
            for _ in range(max_retries):
                pop_data = get_data(api, row['series_id_pop'])
                if pop_data is not None:
                    break
                time.sleep(0.5)

            if pop_data is not None:
                for year in years:
                    if year in pop_data.index.year:
                        CFS_counties.at[idx, f'POP_{year}'] = pop_data.loc[pop_data.index.year == year, 'value'].values[0]/1000

                # Filling empty data for CT counties
                if row['State_USPS'] == 'CT' and row['CNTY_NAME'] in CONNECTICUT_COUNTIES:
                    pop_data.index = pop_data.index.year
                    filled_values = pop_data['value'].reindex(years).ffill().values
                    CFS_counties.loc[idx, [f'POP_{year}' for year in years]] = filled_values/1000

    return CFS_counties

def aggregate_pop_by_cfs_zone(CFS_counties):
    """Aggregate population by CFS zone."""
    aggregated_pop = CFS_counties.groupby(['CFS17_NAME']).agg(
        {f'POP_{year}': 'sum' for year in range(2018, 2024)}
    ).reset_index()
    
    aggregated_pop = aggregated_pop.merge(
        CFS_counties[['CFS17_NAME', 'State_Name', 'CFS17_GEOID']].drop_duplicates(),
        on='CFS17_NAME',
        how='left'
    )    
    return aggregated_pop

In [11]:
CFS_counties_pop = add_pop_to_counties(api, add_series_ids_pop(api, CFS_counties))

In [12]:
CFS_counties_pop[CFS_counties_pop['State_USPS']=='CT']

,ST,CNTY,CNTY_NAME,CFS07_AREA,CFS12_AREA,CFS17_AREA,CFS07_TYPE,CFS12_TYPE,CFS17_TYPE,CFS07_DDESTGEO,...,CFS17_NAME,State_USPS,State_Name,series_id_pop,POP_2018,POP_2019,POP_2020,POP_2021,POP_2022,POP_2023
404,9,1,Fairfield County,408,408,408,C,C,C,92,...,"New York-Newark, NY-NJ-CT-PA CFS Area (CT Part)",CT,Connecticut,CTFAIR1POP,0.945511,0.944388,0.955895,0.959768,0.959768,0.959768
405,9,5,Litchfield County,408,408,408,C,C,C,92,...,"New York-Newark, NY-NJ-CT-PA CFS Area (CT Part)",CT,Connecticut,CTLITC5POP,0.181151,0.180396,0.184874,0.185,0.185,0.185
406,9,9,New Haven County,408,408,408,C,C,C,92,...,"New York-Newark, NY-NJ-CT-PA CFS Area (CT Part)",CT,Connecticut,CTNEWH9POP,0.856895,0.853818,0.863498,0.8637,0.8637,0.8637
636,9,3,Hartford County,278,25540,25540,C,M,M,91,...,"Hartford-West Hartford-East Hartford, CT CFS ...",CT,Connecticut,CTHART3POP,0.892767,0.891349,0.897932,0.896854,0.896854,0.896854
637,9,7,Middlesex County,278,25540,25540,C,M,M,91,...,"Hartford-West Hartford-East Hartford, CT CFS ...",CT,Connecticut,CTMIDD7POP,0.163053,0.162538,0.163923,0.164759,0.164759,0.164759
638,9,13,Tolland County,278,25540,25540,C,M,M,91,...,"Hartford-West Hartford-East Hartford, CT CFS ...",CT,Connecticut,CTTOLL3POP,0.150913,0.150865,0.14965,0.150293,0.150293,0.150293
951,9,11,New London County,99999,99999,99999,R,R,R,99,...,Remainder of Connecticut,CT,Connecticut,None,None,None,None,None,None,None
952,9,15,Windham County,278,99999,99999,C,R,R,91,...,Remainder of Connecticut,CT,Connecticut,None,None,None,None,None,None,None


In [13]:
pop_columns = [f'POP_{year}' for year in range(2018, 2024)]

missing_pop = CFS_counties_pop[
    CFS_counties_pop[pop_columns].isna().any(axis=1) &  # There is at least one NaN in GDP columns
    ~CFS_counties_pop['CFS17_NAME'].str.startswith("Remainder of")  # Skip remainder zones
]

missing_pop[['CNTY_NAME', 'State_Name', 'CFS17_NAME', 'series_id_pop'] + pop_columns]

,CNTY_NAME,State_Name,CFS17_NAME,series_id_pop,POP_2018,POP_2019,POP_2020,POP_2021,POP_2022,POP_2023


In [12]:
# LaSalle County, IL - ILLASA0POP
# Broomfield County, CO - COBROO4POP
# Denver County, CO - CODENV5POP
# Philadelphia County, PA - PAPHIL5POP
# San Francisco County, CA - CASANF0POP
# Honolulu County, HI - HIHONO7POP
# District of Columbia, DC - DCPOP

In [14]:
State_POP = get_state_pop(api, CFS_counties)

In [15]:
State_POP

,State_Name,POP_2018,POP_2019,POP_2020,POP_2021,POP_2022,POP_2023
0,New York,19.544098,19.463131,20.105171,19.848276,19.703747,19.737367
1,Georgia,10.519389,10.628020,10.732888,10.792060,10.931805,11.064432
2,Alabama,4.891628,4.907965,5.033094,5.049196,5.076181,5.117673
3,Massachusetts,6.885720,6.894883,6.994598,7.000474,7.022468,7.066568
4,New Hampshire,1.355064,1.360783,1.378756,1.387677,1.396678,1.402199
5,Rhode Island,1.059338,1.058158,1.096530,1.097246,1.099498,1.103429
6,North Carolina,10.391358,10.501384,10.449652,10.564320,10.710793,10.881189
7,Illinois,12.724685,12.667017,12.799088,12.700641,12.621821,12.642259
8,Indiana,6.698481,6.731010,6.790497,6.815907,6.844545,6.880131
9,Kentucky,4.464273,4.472345,4.508318,4.507583,4.519233,4.550595


In [16]:
CFS_counties_pop.to_csv('CFS_counties_pop.csv', index=None)

In [17]:
CFS_counties_pop = pd.read_csv('CFS_counties_pop.csv')

In [18]:
CFS_agg_pop = aggregate_pop_by_cfs_zone(CFS_counties_pop)
CFS_agg_pop

,CFS17_NAME,POP_2018,POP_2019,POP_2020,POP_2021,POP_2022,POP_2023,State_Name,CFS17_GEOID
0,"Albany-Schenectady, NY CFS Area",1.171398,1.167389,1.190423,1.195607,1.192529,1.192181,New York,E330000US3610400000
1,"Atlanta-Athens-Clarke County-Sandy Springs, GA...",6.630953,6.715360,6.799397,6.850080,6.941268,7.023331,Georgia,E330000US1312200000
2,"Austin-Round Rock, TX CFS Area",2.166805,2.228106,2.300135,2.358634,2.423170,2.473275,Texas,E330000US4899912420
3,"Baltimore-Columbia-Towson, MD CFS Area",2.802908,2.803903,2.842668,2.843219,2.834813,2.834316,Maryland,E330000US2454812580
4,"Baton Rouge, LA CFS Area",0.833221,0.835482,0.849323,0.851950,0.851884,0.853501,Louisiana,E330000US2299912940
...,...,...,...,...,...,...,...,...,...
127,"Virginia Beach-Norfolk, VA-NC CFS Area (VA Part)",1.691978,1.697738,1.725896,1.728074,1.726395,1.727503,Virginia,E330000US5154500000
128,"Washington-Arlington-Alexandria, DC-VA-MD-WV ...",0.704147,0.708253,0.670917,0.669256,0.676725,0.687324,District of Columbia,E330000US1154847900
129,"Washington-Arlington-Alexandria, DC-VA-MD-WV ...",2.469874,2.479693,2.558711,2.556451,2.552257,2.565996,Maryland,E330000US2454847900
130,"Washington-Arlington-Alexandria, DC-VA-MD-WV ...",3.012015,3.038948,3.065891,3.068753,3.078243,3.094948,Virginia,E330000US5154847900


In [19]:
def compute_remainder_pop(state_pop_df, CFS_agg_pop):
    """Calculate population of remainder zones and rename them, if they are only CFS zones in their state."""
    
    CFS_agg_pop = CFS_agg_pop.copy()

    # 1. Population sum by non-remainder zones in each state
    sum_normal_pop = CFS_agg_pop[~CFS_agg_pop['CFS17_NAME'].str.startswith("Remainder of")].groupby('State_Name')[
        [f'POP_{year}' for year in range(2018, 2024)]
    ].sum().reset_index()

    # 2. Merge state populations and calculate population of remainder zones
    remainder_pop = state_pop_df.merge(sum_normal_pop, on='State_Name', how='left', suffixes=('_state', '_sum'))
    
    # Filling NaNs with 0
    for year in range(2018, 2024):
        remainder_pop[f'POP_{year}_sum'] = remainder_pop[f'POP_{year}_sum'].infer_objects(copy=False).fillna(0).astype(float)
    
    for year in range(2018, 2024):
        remainder_pop[f'POP_{year}_remainder'] = remainder_pop[f'POP_{year}_state'] - remainder_pop[f'POP_{year}_sum']

    # 3. Update population of remainder zones in CFS_agg_pop
    for year in range(2018, 2024):
        CFS_agg_pop[f'POP_{year}'] = CFS_agg_pop.apply(
            lambda row: remainder_pop.loc[remainder_pop['State_Name'] == row['State_Name'], f'POP_{year}_remainder'].values[0]
            if row['CFS17_NAME'].startswith("Remainder of") else row[f'POP_{year}'], axis=1
        )

    # 4. Determine states where only one CFS zones (and it is remainder one) and rename it
    state_zone_counts = CFS_agg_pop['State_Name'].value_counts()
    single_cfs_states = state_zone_counts[state_zone_counts == 1].index

    CFS_agg_pop.loc[CFS_agg_pop['State_Name'].isin(single_cfs_states) & CFS_agg_pop['CFS17_NAME'].str.startswith("Remainder of"), 'CFS17_NAME'] = CFS_agg_pop['State_Name']

    # 5. For those state store population data as state population
    for state in single_cfs_states:
        state_pop = state_pop_df.loc[state_pop_df['State_Name'] == state, [f'POP_{year}' for year in range(2018, 2024)]].values[0]
        CFS_agg_pop.loc[CFS_agg_pop['State_Name'] == state, [f'POP_{year}' for year in range(2018, 2024)]] = state_pop

    return CFS_agg_pop

In [21]:
Final_pop = compute_remainder_pop(State_POP, CFS_agg_pop)
Final_pop[Final_pop['State_Name']=='Connecticut']

,CFS17_NAME,POP_2018,POP_2019,POP_2020,POP_2021,POP_2022,POP_2023,State_Name,CFS17_GEOID
30,"Hartford-West Hartford-East Hartford, CT CFS ...",1.206733,1.204752,1.211505,1.211906,1.211906,1.211906,Connecticut,E330000US0927825540
49,"New York-Newark, NY-NJ-CT-PA CFS Area (CT Part)",1.983557,1.978602,2.004267,2.008468,2.008468,2.008468,Connecticut,E330000US0940800000
70,Remainder of Connecticut,0.384271,0.382668,0.364146,0.386233,0.397551,0.422649,Connecticut,E330000US0999999999


In [22]:
Final_pop.to_csv('Final_pop.csv', index=None)

## GDP

In [21]:
def get_series_id_gdp(api, county_name, state_code):
    """Get series_id for county-level GDP data."""
    search_text = f"{county_name}, {state_code} Real Gross Domestic Product"
    title_pattern = f"^Real Gross Domestic Product: All Industries in (?:the )?{county_name}, {state_code}(?:\\s*\\(.*\\))?$"
    return get_series_id(api, search_text, title_pattern, 'Annual', 'Thousands of Chained 2017 U.S. Dollars')

def get_series_id_state_gdp(api, state_name):
    """Get series_id for state-level GDP data."""
    search_text = f"{state_name} Real Gross Domestic Product"
    title_pattern = f"^Real Gross Domestic Product: All Industry Total in (?:the )?{state_name}$"
    return get_series_id(api, search_text, title_pattern, 'Annual', 'Millions of Chained 2017 Dollars')

def add_series_ids_gdp(api, CFS_counties, max_retries=3):
    """Assign GDP series_id with manual overrides for specific counties."""

    # "search-problematic" counties from Virgiia with common GDP data
    VIRGINIA_GROUPS = {
        ('James City County', 'Williamsburg city'): 'REALGDPALL51931',
        ('Poquoson city', 'York County'): 'REALGDPALL51958',
        ('Colonial Heights city', 'Dinwiddie County', 'Petersburg city'): 'REALGDPALL51918',
        ('Prince George County', 'Hopewell city'): 'REALGDPALL51941',
        ('Fairfax city', 'Fairfax County', 'Falls Church city'): 'REALGDPALL51919',
        ('Fredericksburg city', 'Spotsylvania County'): 'REALGDPALL51951',
        ('Manassas city', 'Manassas Park city', 'Prince William County'): 'REALGDPALL51942'
    }

    # Manual series_id for DC
    MANUAL_SERIES = {('District of Columbia', 'DC'): 'DCRGSP'}

    CFS_counties = CFS_counties.copy()
    CFS_counties['series_id_gdp'] = None

    for idx, row in CFS_counties.iterrows():
        if "Remainder of" in row['CFS17_NAME']:
            continue
        
        key = (row['CNTY_NAME'], row['State_USPS'])

        # 1. If a county is in manual series - use given id
        if key in MANUAL_SERIES:
            CFS_counties.at[idx, 'series_id_gdp'] = MANUAL_SERIES[key]
            continue

        # 2. If a county is from Virginia-problematic group counties  — assign common series_id for that "groups"
        for group, series_id in VIRGINIA_GROUPS.items():
            if row['State_USPS'] == 'VA' and row['CNTY_NAME'] in group:
                CFS_counties.at[idx, 'series_id_gdp'] = series_id
                break  # County is found, move to the next
        
        # 3. If a county is not problematic — do simple search and request
        if pd.isna(CFS_counties.at[idx, 'series_id_gdp']):
            series_id = None
            for _ in range(max_retries):
                series_id = get_series_id_gdp(api, row['CNTY_NAME'], row['State_USPS'])
                if series_id:
                    break
                time.sleep(0.5)
            
            CFS_counties.at[idx, 'series_id_gdp'] = series_id

    return CFS_counties

In [22]:
def get_state_gdp(api, CFS_counties, max_retries=3):
    """Get GDP data for each state for each year and store it in a separate DataFrame."""
    state_gdp_data = []
    
    for state_name in CFS_counties['State_Name'].unique():
        series_id = get_series_id_state_gdp(api, state_name)
        if not series_id:
            continue  # Пропускаем, если нет series_id
        
        gdp_data = None
        for _ in range(max_retries):
            gdp_data = get_data(api, series_id)
            if gdp_data is not None:
                break
            time.sleep(0.5)

        if gdp_data is not None:
            state_gdp_data_row = {'State_Name': state_name}
            for year in range(2018, 2024):
                value = gdp_data.loc[gdp_data.index.year == year, 'value']
                state_gdp_data_row[f'GDP_{year}'] = value.iloc[0] if not value.empty else None
            state_gdp_data.append(state_gdp_data_row)

    return pd.DataFrame(state_gdp_data)


def add_gdp_to_counties(api, CFS_counties, CFS_counties_pop, max_retries=3):
    """Get and assign GDP values for counties, distributing group GDP proportionally to population."""

    CFS_counties = CFS_counties.copy()
    years = list(range(2018, 2024))

    for year in years:
        CFS_counties[f'GDP_{year}'] = None

    # Determine which counties belong to "groups"
    grouped_series = CFS_counties['series_id_gdp'].dropna().value_counts()
    grouped_series = grouped_series[grouped_series > 1].index  # Keep only those, that occur more than once

    for series_id in CFS_counties['series_id_gdp'].dropna().unique():
        gdp_data = None
        for _ in range(max_retries):
            gdp_data = get_data(api, series_id)
            if gdp_data is not None:
                break
            time.sleep(0.5)
        
        if gdp_data is None:
            continue

        group = CFS_counties[CFS_counties['series_id_gdp'] == series_id]

        # If series_id belongs to "group" of counties, then divide GDP proportional to population
        if series_id in grouped_series:
            pop_data = CFS_counties_pop.loc[group.index, [f'POP_{year}' for year in years]]
            total_pop = pop_data.sum()  # Общее население группы

            for year in years:
                if year in gdp_data.index.year and total_pop[f'POP_{year}'] > 0:
                    total_gdp = gdp_data.loc[gdp_data.index.year == year, 'value'].values[0] / 1000
                    CFS_counties.loc[group.index, f'GDP_{year}'] = pop_data[f'POP_{year}'] / total_pop[f'POP_{year}'] * total_gdp
        else:
            # Standard case - just store GDP data for normal cases
            for year in years:
                if year in gdp_data.index.year:
                    CFS_counties.loc[group.index, f'GDP_{year}'] = gdp_data.loc[gdp_data.index.year == year, 'value'].values[0] / 1000

    return CFS_counties



def aggregate_gdp_by_cfs_zone(CFS_counties):
    """Aggregate GDP by CFS zone."""
    aggregated_gdp = CFS_counties.groupby(['CFS17_NAME']).agg(
        {f'GDP_{year}': 'sum' for year in range(2018, 2024)}
    ).reset_index()
    
    aggregated_gdp = aggregated_gdp.merge(
        CFS_counties[['CFS17_NAME', 'State_Name', 'CFS17_GEOID']].drop_duplicates(),
        on='CFS17_NAME',
        how='left'
    )    
    return aggregated_gdp

In [23]:
State_GDP = get_state_gdp(api, CFS_counties)

In [24]:
State_GDP

,State_Name,GDP_2018,GDP_2019,GDP_2020,GDP_2021,GDP_2022,GDP_2023
0,New York,1665186.8,1704364.3,1656784.8,1736222.3,1765102.1,1791210.7
1,Georgia,600934.7,620744.9,604745.9,643013.8,665678.1,678201.2
2,Alabama,220808.8,225272.8,222288.8,233726.6,238556.5,245354.7
3,Massachusetts,549262.1,566326.8,559765.0,595911.8,608052.9,615504.5
4,New Hampshire,82269.5,84438.2,83152.6,89949.3,91416.1,93466.7
5,Rhode Island,58740.1,60157.6,58231.7,60803.0,62279.4,63277.0
6,North Carolina,556573.7,567975.0,564504.4,600217.3,619537.2,638067.3
7,Illinois,851517.2,858018.3,810200.9,855888.1,876536.0,885651.3
8,Indiana,368762.9,370507.2,359787.3,387122.0,399280.8,404290.3
9,Kentucky,205940.1,211783.1,206049.6,215133.2,219755.6,224418.0


In [25]:
CFS_counties_GDP = add_gdp_to_counties(api, add_series_ids_gdp(api, CFS_counties), CFS_counties_pop)

In [46]:
CFS_counties_GDP[CFS_counties_GDP['State_USPS']=='IL']

,ST,CNTY,CNTY_NAME,CFS07_AREA,CFS12_AREA,CFS17_AREA,CFS07_TYPE,CFS12_TYPE,CFS17_TYPE,CFS07_DDESTGEO,...,CFS17_NAME,State_USPS,State_Name,series_id_gdp,GDP_2018,GDP_2019,GDP_2020,GDP_2021,GDP_2022,GDP_2023
90,17,11,Bureau County,99999,176,176,R,C,C,179,...,"Chicago-Naperville, IL-IN-WI CFS Area (IL Part)",IL,Illinois,REALGDPALL17011,1204.769,1231.754,1179.257,1263.175,1243.816,1210.687
91,17,31,Cook County,176,176,176,C,C,C,171,...,"Chicago-Naperville, IL-IN-WI CFS Area (IL Part)",IL,Illinois,REALGDPALL17031,407601.05,413294.372,385762.799,407808.153,421205.012,427566.643
92,17,37,DeKalb County,176,176,176,C,C,C,171,...,"Chicago-Naperville, IL-IN-WI CFS Area (IL Part)",IL,Illinois,REALGDPALL17037,4149.597,4144.447,4160.157,4342.386,4218.911,4177.414
93,17,43,DuPage County,176,176,176,C,C,C,171,...,"Chicago-Naperville, IL-IN-WI CFS Area (IL Part)",IL,Illinois,REALGDPALL17043,94519.38,96289.937,92525.418,97299.244,99442.496,100921.668
94,17,63,Grundy County,176,176,176,C,C,C,171,...,"Chicago-Naperville, IL-IN-WI CFS Area (IL Part)",IL,Illinois,REALGDPALL17063,4023.174,4226.97,4301.657,4300.511,4245.114,4536.653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,17,193,White County,99999,99999,99999,R,R,R,179,...,Remainder of Illinois,IL,Illinois,None,None,None,None,None,None,None
1235,17,195,Whiteside County,99999,99999,99999,R,R,R,179,...,Remainder of Illinois,IL,Illinois,None,None,None,None,None,None,None
1236,17,199,Williamson County,99999,99999,99999,R,R,R,179,...,Remainder of Illinois,IL,Illinois,None,None,None,None,None,None,None
1237,17,201,Winnebago County,99999,99999,99999,R,R,R,179,...,Remainder of Illinois,IL,Illinois,None,None,None,None,None,None,None


In [36]:
CFS_counties_GDP.to_csv('CFS_counties_GDP.csv', index=None)

In [45]:
gdp_columns = [f'GDP_{year}' for year in range(2018, 2024)] 

missing_gdp = CFS_counties_GDP[
    CFS_counties_GDP[gdp_columns].isna().any(axis=1) &  
    ~CFS_counties_GDP['CFS17_NAME'].str.startswith("Remainder of")
]

missing_gdp[['CNTY_NAME', 'State_Name', 'CFS17_NAME', 'series_id_gdp'] + gdp_columns]

,CNTY_NAME,State_Name,CFS17_NAME,series_id_gdp,GDP_2018,GDP_2019,GDP_2020,GDP_2021,GDP_2022,GDP_2023


In [28]:
# James City County + Williamsburg city - REALGDPALL51931
# Poquoson city	+ York County - REALGDPALL51958
# Colonial Heights city	+ Dinwiddie County	+ Petersburg city - REALGDPALL51918
# Prince George County + Hopewell city - REALGDPALL51941
# Fairfax city + Fairfax County	+ Falls Church city - REALGDPALL51919
# Fredericksburg city + Spotsylvania County - REALGDPALL51951
# Manassas city	+ Manassas Park city + Prince William County - REALGDPALL51942

# District of Columbia, DC - DCRGSP 

In [29]:
CFS_agg_gdp = aggregate_gdp_by_cfs_zone(CFS_counties_GDP)
CFS_agg_gdp[CFS_agg_gdp['State_Name']=='Alabama']

,CFS17_NAME,GDP_2018,GDP_2019,GDP_2020,GDP_2021,GDP_2022,GDP_2023,State_Name,CFS17_GEOID
6,"Birmingham-Hoover-Talladega, AL CFS Area",71477.836,72438.975,71185.682,75051.076,76462.925,77437.754,Alabama,E330000US0114200000
46,"Mobile-Daphne-Fairhope, AL CFS Area",27823.073,28671.459,28739.163,30636.904,31198.531,32622.927,Alabama,E330000US0138000000
64,Remainder of Alabama,0,0,0,0,0,0,Alabama,E330000US0199999999


In [30]:
def compute_remainder_gdp(state_gdp_df, CFS_agg_gdp):
    """Calculate population of remainder zones  and rename them, if they are only CFS zone in their state."""
    
    CFS_agg_gdp = CFS_agg_gdp.copy()

    # 1. 
    sum_normal_gdp = CFS_agg_gdp[~CFS_agg_gdp['CFS17_NAME'].str.startswith("Remainder of")].groupby('State_Name')[
        [f'GDP_{year}' for year in range(2018, 2024)]
    ].sum().reset_index()

    # 2. 
    remainder_gdp = state_gdp_df.merge(sum_normal_gdp, on='State_Name', how='left', suffixes=('_state', '_sum'))
    
    for year in range(2018, 2024):
        remainder_gdp[f'GDP_{year}_sum'] = remainder_gdp[f'GDP_{year}_sum'].infer_objects(copy=False).fillna(0).astype(float)
        remainder_gdp[f'GDP_{year}_remainder'] = remainder_gdp[f'GDP_{year}_state'] - remainder_gdp[f'GDP_{year}_sum']

    # 3. 
    for year in range(2018, 2024):
        CFS_agg_gdp[f'GDP_{year}'] = CFS_agg_gdp.apply(
            lambda row: remainder_gdp.loc[remainder_gdp['State_Name'] == row['State_Name'], f'GDP_{year}_remainder'].values[0]
            if row['CFS17_NAME'].startswith("Remainder of") else row[f'GDP_{year}'], axis=1
        )

    # 4. 
    state_zone_counts = CFS_agg_gdp['State_Name'].value_counts()
    single_cfs_states = state_zone_counts[state_zone_counts == 1].index

    CFS_agg_gdp.loc[CFS_agg_gdp['State_Name'].isin(single_cfs_states) & CFS_agg_gdp['CFS17_NAME'].str.startswith("Remainder of"), 'CFS17_NAME'] = CFS_agg_gdp['State_Name']

    # 5.
    for state in single_cfs_states:
        state_gdp = state_gdp_df.loc[state_gdp_df['State_Name'] == state, [f'GDP_{year}' for year in range(2018, 2024)]].values[0]
        CFS_agg_gdp.loc[CFS_agg_gdp['State_Name'] == state, [f'GDP_{year}' for year in range(2018, 2024)]] = state_gdp

    return CFS_agg_gdp

In [31]:
Final_gdp = compute_remainder_gdp(State_GDP, CFS_agg_gdp)
Final_gdp[Final_gdp['State_Name']=='Idaho']

,CFS17_NAME,GDP_2018,GDP_2019,GDP_2020,GDP_2021,GDP_2022,GDP_2023,State_Name,CFS17_GEOID
75,Idaho,77652.1,81144.3,82717.4,88969.6,92903.7,95897.9,Idaho,E330000US1699999999


In [32]:
Final_gdp.to_csv('Final_gdp.csv', index=None)

## GDP per Capita (Final)

In [24]:
Final_gdp = pd.read_csv("Final_gdp.csv")
Final_pop = pd.read_csv("Final_pop.csv")

In [25]:
CFS_data = Final_gdp.merge(Final_pop[['CFS17_GEOID', 'POP_2018', 'POP_2019', 'POP_2020', 'POP_2021', 'POP_2022', 'POP_2023']], on = 'CFS17_GEOID', how = 'left')
years = list(range(2018, 2024))

for year in years:
    CFS_data[f'GDP_per_capita_{year}'] = CFS_data[f'GDP_{year}'] / CFS_data[f'POP_{year}']

In [27]:
CFS_data[CFS_data['State_Name']=='Connecticut']

,CFS17_NAME,GDP_2018,GDP_2019,GDP_2020,GDP_2021,GDP_2022,GDP_2023,State_Name,CFS17_GEOID,POP_2018,...,POP_2020,POP_2021,POP_2022,POP_2023,GDP_per_capita_2018,GDP_per_capita_2019,GDP_per_capita_2020,GDP_per_capita_2021,GDP_per_capita_2022,GDP_per_capita_2023
30,"Hartford-West Hartford-East Hartford, CT CFS ...",100888.028,101852.445,94780.867,97641.545,100472.967,102759.900,Connecticut,E330000US0927825540,1.206733,...,1.211505,1.211906,1.211906,1.211906,83604.267058,84542.250189,78233.987478,80568.579576,82904.917543,84791.972315
49,"New York-Newark, NY-NJ-CT-PA CFS Area (CT Part)",149736.082,149014.288,143055.122,149053.593,154549.995,159037.089,Connecticut,E330000US0940800000,1.983557,...,2.004267,2.008468,2.008468,2.008468,75488.671110,75312.916898,71375.281836,74212.580434,76949.194610,79183.282482
70,Remainder of Connecticut,23958.690,24358.667,22929.211,23143.562,23952.938,24831.411,Connecticut,E330000US0999999999,0.384271,...,0.364146,0.386233,0.397551,0.422649,62348.420776,63654.831342,62967.081885,59921.244430,60251.233175,58751.850827


In [28]:
CFS_data.to_csv('CFS_data.csv', index=None)